In [5]:
from torchvision import models
import torch
import torch.nn as nn
from torch.profiler import profile, ProfilerActivity
import torch.distributed as dist
import torch.multiprocessing as mp
from torchsummary import summary
from deepspeed.profiling.flops_profiler import get_model_profile

In [2]:
num_parameters = sum(p.numel() for p in models.resnet18().parameters())
num_parameters

11689512

In [4]:
models.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
with torch.cuda.device(0):
    model = models.vgg16()
    batch_size = 1
    flops, macs, params = get_model_profile(model=model, # model
                                     input_shape=(batch_size, 3, 224, 224), # input shape or input to the input_constructor
                                     print_profile=True, # prints the model graph with the measured profile attached to each module
                                     top_modules=3, # the number of top modules to print aggregated profile
                                     warm_up=3, # the number of warm-ups before measuring the time of each module
    )
    print("{:<30}  {:<8}".format("Batch size: ", batch_size))
    print('{:<30}  {:<8}'.format('Number of MACs: ', macs))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params))
    print('{:<30}  {:<8}'.format('Number of FLOPs: ', flops))


-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 3:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per gpu:                                               138.36 M
params of model = params per GPU * mp_size:                   1       
fwd MACs per GPU:                                             15.47 GMACs
fwd flops per GPU:                                            30.97 G 
fwd flops of model = fwd flops per GPU * mp_size:             1       
fwd latency:                                                  59.17 ms
fwd FLOPS per GPU = fwd flops per GPU / fwd latency:    

In [3]:
import pandas as pd
from pathlib import Path
pd.options.display.expand_frame_repr = False
pd.options.display.max_rows = None
pd.options.display.max_columns = None

df = pd.concat([pd.read_csv(f, sep="\t") for f in Path("bench/results/20220511-220029/").glob("*.csv")], ignore_index=True, sort=False)
df.pivot_table(index=["op", "nchannels"], columns="nthreads", values=["comm_time", "comp_time"], aggfunc="median")

comm_time                                                                                            ... comp_time                                                                                 
nthreads                0         32        64        96        128       160       192       224       256       288  ...       352      384      416      448      480      512      544      576      608      640
op      nchannels                                                                                                      ...                                                                                           
avgpool 0               0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN  ...       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN
        1               0.0  687.8865  368.0450  365.1330  264.6600  216.4655  175.4800  154.6950  140.6185  130.2095  ...    0.9475   0.9520   0.9535   0.9610   0.9460   0.9600   0.9705   0.9650   0.9610   0.9470
        2               0.0  366.1885  195.0055  193.4975  156.8105  110.3795   94.2955   82.6500   75.1580   70.0995  ...    0.9755   0.9695   0.9775   0.9760   0.9965   0.9780   0.9990   0.9710   1.0240   1.0085
        3               0.0  245.7290  132.1600  130.8140   94.8615   76.7330   73.7055   56.9690   58.9530   47.7775  ...    1.0050   0.9920   1.0055   0.9875   0.9705   1.0160   1.0255   1.0225   1.0360   1.0420
        4               0.0  186.8410  100.7140  100.0860   72.5565   57.6265   50.0135   44.1000   40.5375   37.6715  ...    1.0045   1.0155   1.0520   1.0445   1.0340   1.0205   1.0095   0.9970   1.0500   1.0690
        5               0.0  149.7225   81.6620   80.8910   72.3745   47.1215   71.4280   36.9465   33.3760   30.6250  ...    1.0260   1.0615   1.0315   1.0295   1.0345   1.0355   0.9960   1.0760   1.0800   0.9985
        6               0.0  125.2365   68.5855   68.1620  157.1000   44.6525   35.0700   39.6135   48.2530   29.1470  ...    0.9890   1.0370   1.0555   1.0075   1.0455   1.0110   1.0900   1.0470   1.1055   1.1310
        7               0.0  108.9780   59.8290   59.5590   43.9320   35.4300   30.8355   27.4420   62.0355   23.6520  ...    1.0455   1.0435   1.1065   1.0030   1.0900   1.0960   1.1375   1.1370   1.1565   1.0400
        8               0.0  107.2475   52.6700   52.7290   41.1700   31.6975   28.5950   25.9630   24.2740   31.5295  ...    1.0475   1.0735   1.0920   1.0980   1.1135   1.1300   1.0235   1.0210   1.0390   1.0305
        9               0.0  139.4225   47.4235   47.2925   59.6290   29.0175   25.9705   23.7045   25.7860   22.1955  ...    1.1040   1.0095   1.0710   1.0340   1.0635   1.1440   1.1995   1.1300   1.2140   1.1935
        10              0.0   77.5270   44.3190   43.6340   32.1605   26.5405   23.6840   78.9190   91.4715   19.6350  ...    1.0850   1.1070   1.0760   1.0630   1.0650   1.1660   1.0580   1.2105   1.0635   1.2175
        11              0.0   71.8765   41.1995   43.6415   30.5510   24.8450  108.4675  132.9070   20.7220   18.2195  ...    1.1040   1.0825   1.0910   1.0835   1.0860   1.0875   1.2800   1.0795   1.0750   1.2850
        12              0.0   65.8685   63.0845   36.6825   27.7555   23.3520   20.5315   19.1485   26.4725   19.8990  ...    1.0945   1.0970   1.0835   1.0955   1.0975   1.0945   1.2660   1.0820   1.2410   1.2985
        13              0.0   61.8380   34.5435   34.3870   26.7010   21.9260   20.4990   19.8810   18.7195   18.2970  ...    1.2580   1.1245   1.1565   1.1820   1.0885   1.2035   1.3105   1.2820   1.2965   1.2320
        14              0.0   56.8610   64.5730   33.2380   25.2400   21.7185   20.0850   19.7500   19.9965   19.5715  ...    1.1305   1.1245   1.1890   1.1855   1.2125   1.2300   1.1135   1.0485   1.1470   1.1220
        15              0.0   53.1905   31.6900   31.0565   24.5955   20.8800  121.9200   22.8055  117.7290   18.6205  ...    1.1400   1.1220   1.1925   1.1905   1.11

nthreads       0        64       128     256     512
op  nchannels                                       
nop 0          0.0      NaN      NaN     NaN     NaN
    1          NaN  23.8585  16.6105  9.1990  7.3385
    2          NaN  12.6755   9.1275  4.9775  3.9605
    3          NaN   8.5915   6.2380  3.7210  3.0330
    4          NaN   6.6895   4.8350  3.1390  2.7975
    5          NaN   5.4815   4.0735  2.8845  2.6580
    6          NaN   4.6605   3.6675  2.7785  2.6665
    7          NaN   4.1445   3.2525  2.8095  2.6015
    8          NaN   3.7545   3.0675  2.7585  2.5390